## Seguimiento avance municipios CTT

El proceso se organiza en seis etapas secuenciales:

1. Parámetros
2. Autenticación OneDrive
3. Consolidación de municipios y etapas
4. Cálculos del avance
5. Reporte avance consolidado
6. Guardar resultados

# **1. Parámetros**

In [ ]:
#### 1. INPUTS ####
#### 1.1 ENLACE DE LA CARPETA DE SEGUIMIENTO CTT
link = "https://planeacionnacional.sharepoint.com/sites/PCCM_UnidadGestionDNP/CoordTec/Forms/AllItems.aspx?id=%2Fsites%2FPCCM%5FUnidadGestionDNP%2FCoordTec%2F10%20Estrategia%5FFT%2F11%5FFT%2FSeguimiento&p=true&ct=1765672075225&or=Teams%2DHL&ga=1&LOF=1"
#### 1.2 CARPETA DONDE SE ENCUENTRAN LOS REPORTES CTT POR MUNICPIO
carpeta_reportes = "Reportes"
#### 1.3 NOMBRE DEL ARCHIVO EN EXCEL QUE CONTIENE LOS CRITERIOS Y LISTAS DE ESTRATEGIA POR MUNICIPIO
excel_criterios_estrategia='CONSOLIDACION SEGUIMIENTO FT v0.xlsx'

#### 2. OUTPUTS ####
ruta_salida = "Consolidado_Municipios.xlsx"

print('Parámetros cargados exitosamente')

Parámetros cargados exitosamente


# **2. Autenticación OneDrive**

In [ ]:
import base64
import msal
import requests

# ==== 1. Datos base ====
CLIENT_ID = "d3590ed6-52b3-4102-aeff-aad2292ab01c"  # App pública de Microsoft
AUTHORITY = "https://login.microsoftonline.com/common"
SCOPE = ["Files.Read.All"]

# ==== 2. Iniciar flujo interactivo con MFA ====
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
print(flow["message"])  # -> Abre la URL que muestra y pega el código

result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception("Error al autenticar:", result.get("error_description"))

token = result["access_token"]
print("Autenticación exitosa ✅")


# Codificar el link para Graph API
encoded_link = base64.urlsafe_b64encode(link.encode()).decode().rstrip("=")

# Endpoint para listar archivos
url = f"https://graph.microsoft.com/v1.0/shares/u!{encoded_link}/driveItem/children"

headers = {"Authorization": f"Bearer {token}"}


metadata_url = f"https://graph.microsoft.com/v1.0/shares/u!{encoded_link}/driveItem"
meta_resp = requests.get(metadata_url, headers=headers).json()

drive_id = meta_resp["parentReference"]["driveId"]
item_id = meta_resp["id"]

# =========================
# 3. Función para listar hijos
# =========================
def listar_hijos(drive_id, item_id):
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/children"
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print("Error al listar hijos:", resp.text)
        return []
    return resp.json().get("value", [])

# =========================
# 4. Recorrer recursivamente solo archivos .xlsx
# =========================
def recorrer_recursivo(drive_id, item_id, ruta_base=""):
    archivos = []
    items = listar_hijos(drive_id, item_id)

    for item in items:
        nombre = item["name"]
        nueva_ruta = f"{ruta_base}/{nombre}" if ruta_base else nombre

        if "folder" in item:
            # Recursión para subcarpetas
            archivos.extend(recorrer_recursivo(drive_id, item["id"], nueva_ruta))
        else:
            # Filtrar solo archivos .xlsx
            if nombre.lower().endswith(".xlsx"):
                archivos.append({
                    "nombre": nombre,
                    "ruta": nueva_ruta,
                    "id": item["id"]
                })
    return archivos

todos_los_archivos = recorrer_recursivo(drive_id, item_id)

def filtrar_por_carpeta(lista, carpeta_objetivo):
    return [
        archivo for archivo in lista
        if archivo["ruta"].startswith(carpeta_objetivo)
    ]



# Filtrar archivos de cada carpeta
reportes = filtrar_por_carpeta(todos_los_archivos, carpeta_reportes)
reportes

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AK7YLN9D2 to authenticate.


# **3. Consolidación de municipios y etapas**

In [ ]:
import requests, io, time, os
import pandas as pd
from io import BytesIO

# ---------------------------
# Helpers de subida a OneDrive
# ---------------------------
def df_a_excel_bytes(df, nombre_hoja="Datos"):
    buf = BytesIO()
    with pd.ExcelWriter(buf, engine="xlsxwriter") as writer:
        df.to_excel(writer, index=False, sheet_name=nombre_hoja)
    buf.seek(0)
    return buf

def subir_excel_pequeno(drive_id, parent_item_id, nombre_archivo, bytes_io, headers):
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{parent_item_id}:/{nombre_archivo}:/content"
    params = {"@microsoft.graph.conflictBehavior": "replace"}  # o "rename"
    r = requests.put(url, headers=headers, params=params, data=bytes_io.read())
    if r.status_code not in (200, 201):
        raise RuntimeError(f"Error subiendo archivo: {r.status_code} - {r.text}")
    return r.json()

def crear_upload_session(drive_id, parent_item_id, nombre_archivo, headers):
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{parent_item_id}:/{nombre_archivo}:/createUploadSession"
    payload = {"item": {"@microsoft.graph.conflictBehavior": "replace", "name": nombre_archivo}}
    r = requests.post(url, headers={**headers, "Content-Type":"application/json"}, json=payload)
    if r.status_code not in (200, 201):
        raise RuntimeError(f"Error creando upload session: {r.status_code} - {r.text}")
    return r.json()["uploadUrl"]

def subir_excel_grande(drive_id, parent_item_id, nombre_archivo, bytes_io, headers, chunk_size=5*1024*1024):
    upload_url = crear_upload_session(drive_id, parent_item_id, nombre_archivo, headers)
    data = bytes_io.getbuffer()
    size = len(data)
    start = 0
    while start < size:
        end = min(start + chunk_size, size) - 1
        chunk = data[start:end+1]
        r = requests.put(
            upload_url,
            headers={
                "Content-Length": str(len(chunk)),
                "Content-Range": f"bytes {start}-{end}/{size}",
            },
            data=chunk
        )
        if r.status_code in (200, 201):  # completado
            return r.json()
        if r.status_code != 202:         # 202 = continuar
            raise RuntimeError(f"Error en chunk upload: {r.status_code} - {r.text}")
        start = end + 1
    raise RuntimeError("Subida no completada (inesperado)")

def guardar_df_en_onedrive(df, drive_id, parent_item_id, headers, nombre_base="consolidado", hoja="Datos"):
    ts = time.strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"{nombre_base}_{ts}.xlsx"
    #nombre_archivo = f"{nombre_base}.xlsx"
    buf = df_a_excel_bytes(df, nombre_hoja=hoja)
    size = len(buf.getbuffer())
    if size <= 4*1024*1024:
        buf.seek(0)
        resp = subir_excel_pequeno(drive_id, parent_item_id, nombre_archivo, buf, headers)
    else:
        buf.seek(0)
        resp = subir_excel_grande(drive_id, parent_item_id, nombre_archivo, buf, headers)
    print(f"✅ Subido: {resp['name']}  (ID: {resp['id']})")
    if "webUrl" in resp:
        print("🔗 Abre aquí:", resp["webUrl"])
    return resp

# -----------------------------------
# Descarga segura con reintentos (opcional)
# -----------------------------------
def descargar_excel_item(drive_id, archivo_id, headers, retries=3, timeout=60):
    download_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{archivo_id}/content"
    intento = 0
    while intento < retries:
        resp = requests.get(download_url, headers=headers, timeout=timeout)
        if resp.status_code == 200:
            return io.BytesIO(resp.content)
        intento += 1
        time.sleep(1 + intento)  # backoff simple
    raise RuntimeError(f"No se pudo descargar item {archivo_id}. Último status: {resp.status_code} - {resp.text}")

# -----------------------------------
# Consolidar hojas de un Excel en memoria
# -----------------------------------
def consolidar_hojas_excel(contenido_excel, nombre_archivo, ruta_origen):
    """Lee todas las hojas y devuelve un DF consolidado por archivo.
       Agrega: Archivo_Origen, Ruta_Origen, Hoja_Origen."""
    try:
        hojas = pd.read_excel(contenido_excel, sheet_name=None)  # None = todas las hojas
    except Exception as e:
        print(f"⚠️ Error leyendo {nombre_archivo}: {e}")
        return pd.DataFrame()

    dfs = []
    for nombre_hoja, df_h in hojas.items():
        if isinstance(df_h, pd.DataFrame) and not df_h.empty:
            df_tmp = df_h.copy()
            df_tmp["Archivo_Origen"] = nombre_archivo
            df_tmp["Ruta_Origen"] = ruta_origen
            df_tmp["Hoja_Origen"] = str(nombre_hoja)
            dfs.append(df_tmp)
    if dfs:
        return pd.concat(dfs, ignore_index=True, sort=False)  # outer concat
    return pd.DataFrame()

# ==========================================
# 🚀 Pipeline principal usando todos_los_archivos
# ==========================================
consolidados_por_archivo = []
total_archivos=len(reportes)
print(f"🔎 Procesando {total_archivos} archivos .xlsx (mun_xlsx)...")

list_cols=['Zona', 'Departamento', 'Municipio', 'Etapa', 'Hito', 'Actividad',       'Acción', 'Observaciones / recomendaciones', 'Fecha inicio',
       'Fecha fin', 'Fecha de ejecución', 'Estado de avance de la actividad',       'Porcentaje de avance', 'Link de soporte', 'Responsable']

for i, archivo in enumerate(reportes, start=1):
    archivo_id  = archivo["id"]
    nombre      = archivo["nombre"]
    ruta_remota = archivo["ruta"]

    # Descargar
    try:
        contenido = descargar_excel_item(drive_id, archivo_id, headers)
    except Exception as e:
        print(f"❌ [{i}/{total_archivos}] Error descargando '{ruta_remota}': {e}")
        continue

    # Consolidar hojas de este libro
    df_archivo = consolidar_hojas_excel(contenido, nombre_archivo=nombre, ruta_origen=ruta_remota)
    if df_archivo.empty:
        print(f"⚠️ [{i}/{total_archivos}] '{ruta_remota}' sin datos utilizables (hojas vacías o error de lectura).")
        continue

    consolidados_por_archivo.append(df_archivo)
    print(f"✅ [{i}/{total_archivos}] Consolidado '{ruta_remota}' -> filas: {len(df_archivo)}")

# Consolidado global
if consolidados_por_archivo:
    df_total = pd.concat(consolidados_por_archivo, ignore_index=True, sort=False)
    df_total.dropna(how="all", inplace=True)  # limpia filas totalmente vacías

    #DF filtrado
    df_total=df_total[list_cols]

    #Limpieza final
    columnas_clave = ["Departamento", "Municipio", "Zona", "Etapa", "Hito"]

    # Quitar espacios en blanco y convertir cadenas vacías en NaN
    df_total[columnas_clave] = df_total[columnas_clave].apply(
        lambda x: x.astype(str).str.strip().replace({"": None, "nan": None})
    )

    # Eliminar filas donde cualquiera de esas columnas esté vacía
    df_total = df_total.dropna(subset=columnas_clave, how="any")

    print("\n📦 RESUMEN CONSOLIDADO")
    print(f"   Archivos consolidados: {len(consolidados_por_archivo)}")
    print(f"   Filas totales: {len(df_total):,}")
    print(f"   Columnas totales: {len(df_total.columns)}", df_total.columns)

    # Subir a OneDrive (misma carpeta a la que estás conectado)
    #_ = guardar_df_en_onedrive(
    #    df_total,        drive_id=drive_id,        parent_item_id=item_id,              # usa una subcarpeta si prefieres
    #    headers=headers,        nombre_base="Consolidado_zona",  # prefijo del archivo
    #    hoja="Consolidado"    )
else:
    print("⚠️ No se consolidó ningún archivo (todos fallaron o estaban vacíos).")


🔎 Procesando 1 archivos .xlsx (mun_xlsx)...


c:\Users\Sebastian\anaconda3\envs\ds_311_gym\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Sebastian\anaconda3\envs\ds_311_gym\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Sebastian\anaconda3\envs\ds_311_gym\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\Sebastian\anaconda3\envs\ds_311_gym\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


✅ [1/1] Consolidado 'Reportes/Reporte seguimiento municipios operacion FT 2025.xlsx' -> filas: 2808

📦 RESUMEN CONSOLIDADO
   Archivos consolidados: 1
   Filas totales: 1,507
   Columnas totales: 15 Index(['Zona', 'Departamento', 'Municipio', 'Etapa', 'Hito', 'Actividad',
       'Acción', 'Observaciones / recomendaciones', 'Fecha inicio',
       'Fecha fin', 'Fecha de ejecución', 'Estado de avance de la actividad',
       'Porcentaje de avance', 'Link de soporte', 'Responsable'],
      dtype='object')


C:\Users\Sebastian\AppData\Local\Temp\ipykernel_5288\50913537.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dfs, ignore_index=True, sort=False)  # outer concat


# **4. Cálculos del avance**

In [ ]:
import io, requests, pandas as pd


#Archivo que contiene los criterios y las listas de municipios con la estrategia
#Debe estar en la misma carpeta compartida
item0 = [f for f in todos_los_archivos if f["nombre"].startswith(excel_criterios_estrategia)][0]
url0  = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item0['id']}/content"

resp = requests.get(url0, headers=headers, allow_redirects=True)
resp.raise_for_status()

# Lee la hoja 'Criterio puntajes' con encabezados en la fila 3
df_crit = pd.read_excel(
    io.BytesIO(resp.content),
    sheet_name="Criterio puntajes")

df_estrategia = pd.read_excel(
    io.BytesIO(resp.content),
    sheet_name="Listas Municipio")

#print(item0["nombre"], df_crit.shape)
#df_crit.head()

In [ ]:
# 1️⃣ Renombrar columnas en df_estrategia para que coincidan con df_total
df_estrategia_ren = df_estrategia.rename(columns={
    'Depto': 'Departamento',
    'Ciudad   ': 'Municipio'
})

# 2️⃣ Seleccionar solo las columnas necesarias del primer DF
df_estrategia_sel = df_estrategia_ren[['divipola', 'Departamento', 'Municipio',
                                      'Estrategia', 'prio']]

# 3️⃣ Hacer el merge con df_total usando left o inner según necesites
df = df_total.merge(
    df_estrategia_sel,
    on=['Departamento', 'Municipio'],
    how='left'   # Cambia a 'inner' si solo quieres coincidencias exactas
)

import numpy as np

# Columna sea tipo datetime
df['Fecha de ejecución'] = pd.to_datetime(df['Fecha de ejecución'], errors='coerce')

# Crear la columna "Fecha mes año" (formato "abr-2024")
df['Fecha mes año'] = df['Fecha de ejecución'].dt.strftime('%b-%Y').fillna('')

#Traer los pesos por actividad
# --- 1) Normalizar claves (evita fallos por espacios o mayúsculas) ---
for d in (df, df_crit):
    for col in ["Etapa", "Hito", "Actividad"]:
        if col in d.columns:  # en df_crit 'Etapa' es 'ETAPA'
            d[col] = d[col].astype(str).str.strip()
    if "ETAPA" in d.columns:
        d["ETAPA"] = d["ETAPA"].astype(str).str.strip()

# --- 2) Asegurar que los puntajes en df_crit sean numéricos (quita % y comas) ---
def to_num(s):
    return pd.to_numeric(
        pd.Series(s).astype(str)
        .str.replace("%", "", regex=False)
        .str.replace(",", ".", regex=False),
        errors="coerce"
    )

for col in ["Ptj", "Ptj CTT", "Ptj FT", "Ptj CTT.1", "Ptj FT.1"]:
    if col in df_crit.columns:
        df_crit[col] = to_num(df_crit[col])

# --- 3) Construir diccionarios de mapeo desde df_crit ---
# Etapa -> Ptj
map_etapa = (df_crit
             .dropna(subset=["ETAPA", "Ptj"])
             .drop_duplicates("ETAPA")
             .set_index("ETAPA")["Ptj"]
             .to_dict())

# Hito -> Ptj CTT / Ptj FT
map_hito_ctt = (df_crit
                .dropna(subset=["Hito", "Ptj CTT"])
                .drop_duplicates("Hito")
                .set_index("Hito")["Ptj CTT"]
                .to_dict())

map_hito_ft  = (df_crit
                .dropna(subset=["Hito", "Ptj FT"])
                .drop_duplicates("Hito")
                .set_index("Hito")["Ptj FT"]
                .to_dict())

# Actividad -> Ptj CTT.1 / Ptj FT.1
map_act_ctt = (df_crit
               .dropna(subset=["Actividad", "Ptj CTT.1"])
               .drop_duplicates("Actividad")
               .set_index("Actividad")["Ptj CTT.1"]
               .to_dict())

map_act_ft  = (df_crit
               .dropna(subset=["Actividad", "Ptj FT.1"])
               .drop_duplicates("Actividad")
               .set_index("Actividad")["Ptj FT.1"]
               .to_dict())

# --- 4) Crear columnas en df usando map ---
df["Ptj_etapa"]           = df["Etapa"].map(map_etapa).fillna(0)
df["Ptj_hito"]            = df["Hito"].map(map_hito_ctt).fillna(0)
df["Ptj_ft_hito"]         = df["Hito"].map(map_hito_ft).fillna(0)
df["Ptj_actividad"]       = df["Actividad"].map(map_act_ctt).fillna(0)
df["Ptj_ft_actividad"]    = df["Actividad"].map(map_act_ft).fillna(0)

# (Opcional) si quieres que queden como porcentajes texto:
# for c in ["Ptj_etapa","Ptj_hito","Ptj_ft_hito","Ptj_actividad","Ptj_ft_actividad"]:
#     df[c] = df[c].round(2).astype(str) + "%"



df['Peso actividad en el hito'] = np.where(
    df['Estrategia'] == 'CTT',
    df['Ptj_actividad'],
    df['Ptj_ft_actividad']
)

# Convertir las dos columnas a numéricas
df["Porcentaje de avance"] = pd.to_numeric(df["Porcentaje de avance"], errors="coerce")
df["Peso actividad en el hito"] = pd.to_numeric(df["Peso actividad en el hito"], errors="coerce")

# Calcular el avance (idéntico a =O2*R2)
df["Avance de la acción dentro de la actividad"] = (
    df["Porcentaje de avance"] * df["Peso actividad en el hito"])
# --- 0) Fecha y númerico
df["Fecha de ejecución"] = pd.to_datetime(df["Fecha de ejecución"], errors="coerce")
df["Avance de la acción dentro de la actividad"] = pd.to_numeric(
    df["Avance de la acción dentro de la actividad"], errors="coerce"
)

cols_base = ["Departamento","Municipio","Estrategia","Etapa","Hito","Actividad"]
df=df.loc[~df.duplicated(subset=cols_base, keep="first")]


#EXPORTAR

print(df.shape)
df.columns

#Filtrar las columnas que se quieren exportar
df1 = df.loc[:, ~df.columns.str.startswith("Ptj")]
df1.head()


(1425, 26)


,Zona,Departamento,Municipio,Etapa,Hito,Actividad,Acción,Observaciones / recomendaciones,Fecha inicio,Fecha fin,...,Estado de avance de la actividad,Porcentaje de avance,Link de soporte,Responsable,divipola,Estrategia,prio,Fecha mes año,Peso actividad en el hito,Avance de la acción dentro de la actividad
0,Zona 3,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.1_Diálogo_y_acuerdos_con_gobernaciones,1.1.1 Socialización proyecto en diálogo con go...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,13647.0,FT - BM,21,,0.0,NaN
1,Zona 3,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.2_Manifestación_de_interés_departamental,1.2.1 Manifestación de interes a nivel departa...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,13647.0,FT - BM,21,,0.0,NaN
2,Zona 3,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.3_Articulación_a_nivel_municipal,1.3.1 Articulación a nivel municipal,Aproximación territorial / nivel de interlocuc...,NaN,NaN,NaN,...,Finalizada,1.0,NaN,Eida Garzón,13647.0,FT - BM,21,Oct-2024,0.5,0.5
3,Zona 3,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.4_Manifestación_de_interes_a_nivel_municipal,1.4.1 MI Respuesta municipio,Se decide aceptar el correo enviado por el mun...,Enviada para firma el 24/10/2024. Se recibio c...,NaN,NaN,...,Finalizada,1.0,NaN,Eida Garzón,13647.0,FT - BM,21,Nov-2024,0.5,0.5
4,Zona 3,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.5_Suscripción_convenios_Departamental,1.5.1 Suscripción convenios Departamental,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,13647.0,FT - BM,21,,0.0,NaN


# **5. Reporte avance consolidado**

In [ ]:
# --- 0) Fecha y númerico
df["Fecha de ejecución"] = pd.to_datetime(df["Fecha de ejecución"], errors="coerce")
df["Avance de la acción dentro de la actividad"] = pd.to_numeric(
    df["Avance de la acción dentro de la actividad"], errors="coerce"
)

# --- 1) UNICOS(Departamento:Actividad) -> combinaciones únicas base ---
cols_base = ["Departamento","Municipio","Estrategia","Etapa","Hito","Actividad"]
df_consolidado = df[cols_base].drop_duplicates().copy()

# --- 2) MAX.SI.CONJUNTO Fecha de ejecución por (Municipio, Actividad) ---
max_fecha = (
    df.groupby(["Municipio","Actividad"], as_index=False)["Fecha de ejecución"]
      .max()
      .rename(columns={"Fecha de ejecución":"Fecha"})
)

# --- 3) MAX.SI.CONJUNTO Avance por (Municipio, Actividad) ---
max_avance = (
    df.groupby(["Municipio","Actividad"], as_index=False)["Avance de la acción dentro de la actividad"]
      .max())


# --- 4) Unir a la tabla consolidada (BUSCARV por Municipio+Actividad) ---
df_consolidado = (
    df_consolidado
    .merge(max_fecha, on=["Municipio","Actividad"], how="left")
    .merge(max_avance, on=["Municipio","Actividad"], how="left")
)

# --- 5) (Opcional) Columna "Fecha mes año" como en tu reporte ---
mes_abbr_es = {1:"ene",2:"feb",3:"mar",4:"abr",5:"may",6:"jun",7:"jul",8:"ago",9:"sep",10:"oct",11:"nov",12:"dic"}
df_consolidado["Fecha mes año"] = df_consolidado["Fecha"].apply(
    lambda d: "" if pd.isna(d) else f"{mes_abbr_es[d.month]}-{d.year}"
)

# (Opcional) ordenar columnas de salida
orden = ["Departamento","Municipio","Estrategia","Etapa","Hito","Actividad","Fecha","Fecha mes año","Avance de la acción dentro de la actividad"]
df_consolidado = df_consolidado[orden]
df_consolidado

,Departamento,Municipio,Estrategia,Etapa,Hito,Actividad,Fecha,Fecha mes año,Avance de la acción dentro de la actividad
0,BOLÍVAR,SAN ESTANISLAO,FT - BM,_1.Aproximación,_1.1_Diálogo_y_acuerdos_con_gobernaciones,1.1.1 Socialización proyecto en diálogo con go...,NaT,,NaN
1,BOLÍVAR,SAN ESTANISLAO,FT - BM,_1.Aproximación,_1.2_Manifestación_de_interés_departamental,1.2.1 Manifestación de interes a nivel departa...,NaT,,NaN
2,BOLÍVAR,SAN ESTANISLAO,FT - BM,_1.Aproximación,_1.3_Articulación_a_nivel_municipal,1.3.1 Articulación a nivel municipal,2024-10-16,oct-2024,0.5
3,BOLÍVAR,SAN ESTANISLAO,FT - BM,_1.Aproximación,_1.4_Manifestación_de_interes_a_nivel_municipal,1.4.1 MI Respuesta municipio,2024-11-28,nov-2024,0.5
4,BOLÍVAR,SAN ESTANISLAO,FT - BM,_1.Aproximación,_1.5_Suscripción_convenios_Departamental,1.5.1 Suscripción convenios Departamental,NaT,,NaN
...,...,...,...,...,...,...,...,...,...
1420,SUCRE,MORROA,FT - BM,_5.Evaluación,_5.2_Evaluación_con_el_cálculo_de_los_resultad...,5.2.1 Evaluación con el cálculo de los resulta...,NaT,,NaN
1421,SUCRE,MORROA,FT - BM,_5.Evaluación,_5.3_Realización_de_propuesta_de_Plan_de_forta...,5.3.1 Elaboración del documento de evaluación,NaT,,NaN
1422,SUCRE,MORROA,FT - BM,_5.Evaluación,_5.4_Validación_y_aseguramiento_de_la_calidad_...,5.4.1 Revisión validadores,NaT,,NaN
1423,SUCRE,MORROA,FT - BM,_5.Evaluación,_5.4_Validación_y_aseguramiento_de_la_calidad_...,5.4.2 Visto bueno especialista,NaT,,NaN


In [ ]:

# Suma tipo SUMAR.SI.CONJUNTO para todas las actividades
df_actividad = (
    df.groupby(["Departamento","Municipio","Actividad"], as_index=False)
      ["Avance de la acción dentro de la actividad"].sum())

# Suma tipo SUMAR.SI.CONJUNTO para todas las actividades
df_hito = (
    df.groupby(["Departamento","Municipio","Hito"], as_index=False)
      ["Avance de la acción dentro de la actividad"].sum()
      .rename(columns={"Avance de la acción dentro de la actividad": "Avance Hito"})
)

df_etapa = (
    df.groupby(["Departamento","Municipio","Etapa"], as_index=False)
      ["Avance de la acción dentro de la actividad"].sum()
      .rename(columns={"Avance de la acción dentro de la actividad": "Avance Etapa"})
)


df_etapa_hito= (
    df.groupby(["Departamento","Municipio","Etapa","Hito"], as_index=False)
      ["Avance de la acción dentro de la actividad"].sum())
df_etapa_hito

,Departamento,Municipio,Etapa,Hito,Avance de la acción dentro de la actividad
0,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.1_Diálogo_y_acuerdos_con_gobernaciones,0.0
1,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.2_Manifestación_de_interés_departamental,0.0
2,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.3_Articulación_a_nivel_municipal,0.5
3,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.4_Manifestación_de_interes_a_nivel_municipal,0.5
4,BOLÍVAR,SAN ESTANISLAO,_1.Aproximación,_1.5_Suscripción_convenios_Departamental,0.0
...,...,...,...,...,...
545,SUCRE,MORROA,_3.Implementación,_3.7_Implementación_Participación,0.0
546,SUCRE,MORROA,_5.Evaluación,_5.1_Levantamiento_de_evaluación_de_capacidade...,0.0
547,SUCRE,MORROA,_5.Evaluación,_5.2_Evaluación_con_el_cálculo_de_los_resultad...,0.0
548,SUCRE,MORROA,_5.Evaluación,_5.3_Realización_de_propuesta_de_Plan_de_forta...,0.0


In [ ]:
# Copia del df original para evitar modificarlo
df2 = df_etapa_hito.copy()

# Crear una columna combinada Etapa + Hito para que sea el nombre de la nueva columna
#df2["Etapa_Hito"] = df2["Etapa"] + "__" + df2["Hito"]

# Hacer pivot (formato ancho)
df_wide = df2.pivot_table(
    index=["Departamento", "Municipio"],
    columns="Hito",
    values="Avance de la acción dentro de la actividad",
    aggfunc="max",  # por si hay duplicados
    fill_value=0
).reset_index()

# Ordenar columnas opcionalmente
df_wide = df_wide.sort_index(axis=1)


# ====== 1. Diccionario de pesos ======
pesos = pd.Series({
    "_1.Aproximación": 0.15,
    "_2.Alistamiento": 0.30,
    "_3.Implementación": 0.40,
    "_5.Evaluación":   0.15,
}, name="peso")

df_final = df_wide.copy()

# ====== 2. Crear columnas agregadas por etapa ======

for etapa in pesos.index:

    # Número del prefijo, ejemplo "_1."
    prefijo = etapa.split(".")[0].replace("_", "")  # → "1" o "2" o "3" o "5"
    prefijo = "_" + prefijo + "."

    # Columnas que corresponden a esa etapa
    cols_etapa = [c for c in df_final.columns if c.startswith(prefijo)]

    # Crear nueva columna con el nombre EXACTO del diccionario
    df_final[etapa] = df_final[cols_etapa].sum(axis=1)


# ====== 3. Calcular Avance Municipio (ponderado) ======
df_final["Avance total"] = 0

for etapa, peso in pesos.items():
    df_final["Avance total"] += df_final[etapa] * peso


# ====== 4. Resultado ======
df_final.head()


Hito,Comités_Operativos_de_Campo,Departamento,Municipio,_1.1_Diálogo_y_acuerdos_con_gobernaciones,_1.2_Manifestación_de_interés_departamental,_1.3_Articulación_a_nivel_municipal,_1.4_Manifestación_de_interes_a_nivel_municipal,_1.5_Suscripción_convenios_Departamental,_1.6_Suscripción_convenios_Municipal,_2.1_Levantamiento_diagnóstico_capacidades_municipales,...,_3.7_Implementación_Participación,_5.1_Levantamiento_de_evaluación_de_capacidades_municipales,_5.2_Evaluación_con_el_cálculo_de_los_resultados_de_las_dimensiones,_5.3_Realización_de_propuesta_de_Plan_de_fortalecimiento,_5.4_Validación_y_aseguramiento_de_la_calidad_del_diagnóstico,_1.Aproximación,_2.Alistamiento,_3.Implementación,_5.Evaluación,Avance total
0,0.0,BOLÍVAR,SAN ESTANISLAO,0.0,0.0,0.5,0.5,0.0,0.0,0.52,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.176136,0.0,0.520455
1,0.0,BOLÍVAR,ZAMBRANO,0.0,0.0,0.5,0.5,0.0,0.0,0.52,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.226136,0.0,0.540455
2,0.0,BOYACÁ,BOAVITA,0.0,0.0,0.5,0.5,0.0,0.0,0.52,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.162500,0.0,0.515000
3,0.0,BOYACÁ,CERINZA,0.0,0.0,0.5,0.5,0.0,0.0,0.52,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.125000,0.0,0.500000
4,0.0,BOYACÁ,COVARACHÍA,0.0,0.0,0.5,0.5,0.0,0.0,0.52,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.125000,0.0,0.500000


# **6. Guardar resultados**

In [ ]:
import io, time


# 1️⃣ Seleccionar solo las columnas que necesitamos del primer DF
list_cols=['Departamento', 'Municipio', 'divipola', 'prio', 'Zona', 'Estrategia']
df1_sel = df1[list_cols]
df1_sel=df1_sel.drop_duplicates(subset=list_cols)

# 2️⃣ Unir el df1 con df_final usando las llaves
df_merged = df_final.merge(
    df1_sel,
    on=['Departamento', 'Municipio'],
    how='left')

# 3️⃣ Renombrar columnas según lo solicitado
df_final = df_merged.rename(columns={
    'Departamento': 'Depto',
    'Municipio': 'Ciudad'})

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:

    df1.to_excel(writer, sheet_name="BaseDatos", index=False)
    df_consolidado.to_excel(writer, sheet_name="Consolidacion", index=False)
    df_final.to_excel(writer, sheet_name="AvanceMunicipio", index=False)


drive_id = meta_resp['parentReference']['driveId']
parent_item_id_Seguimiento = meta_resp['id']

drive_id = 'b!QL4odV8hhEeBbNKcb40OZWQJZRBrLcVGj0PZ8nGt8SPhb4gUjkZ5R7lCfltswdsG'
parent_item_id_Seguimiento = '01D4XMQPYAPXIFKETYYJEJXJ3AUHABO67N'

# 1. Leer archivo ya creado en el disco local
buf = io.BytesIO(open(ruta_salida, "rb").read())

# 2. Tamaño para decidir tipo de subida
size = len(buf.getbuffer())

# 4. Subir a la carpeta Seguimiento usando tus funciones
if size <= 4*1024*1024:
    buf.seek(0)
    resp = subir_excel_pequeno(drive_id, parent_item_id_Seguimiento, ruta_salida, buf, headers)
else:
    buf.seek(0)
    resp = subir_excel_grande(drive_id, parent_item_id_Seguimiento, ruta_salida, buf, headers)

print("✅ Subido correctamente:", resp["name"])
print("🌐 URL:", resp.get("webUrl"))


✅ Subido correctamente: Consolidado_Municipios.xlsx
🌐 URL: https://planeacionnacional.sharepoint.com/sites/PCCM_UnidadGestionDNP/_layouts/15/Doc.aspx?sourcedoc=%7B86C98815-4C50-42F8-B845-CAB9DD727B6D%7D&file=Consolidado_Municipios.xlsx&action=default&mobileredirect=true
